In [1]:
!pip install -q transformers einops accelerate langchain bitsandbytes peft trl

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.6/44.6 kB 548.5 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 265.7/265.7 kB 3.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 25.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.6/92.6 MB 2.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 174.7/174.7 kB 25.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 133.9/133.9 kB 20.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 177.6/177.6 kB 25.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.1/48.1 kB 7.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 521.2/521.2 kB 54.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 100.9/100.9 kB 14.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 7.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 16.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━

In [2]:
!huggingface-cli login

Token will not been saved to git credential helper. Pass `add_to_git_credential=True` if you want to set the git credential as well.
Token is valid (permission: read).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [3]:
!pip install InstructorEmbedding sentence-transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 1.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 13.7 MB/s eta 0:00:00
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125923 sha256=abad3c2f61a35adfac57845588a19c51606db64cf4dfbb832a5a731cf0afe00d
  Stored in directory: /root/.cache/pip/wheels/62/f2/10/1e606fd5f02395388f74e7462910fe851042f97238cbbd902f
Successfully built sentence-transformers


In [4]:
from InstructorEmbedding import INSTRUCTOR
model = INSTRUCTOR('hkunlp/instructor-large')

/usr/local/lib/python3.10/dist-packages/InstructorEmbedding/instructor.py:7: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import trange


.gitattributes:   0%|          | 0.00/1.48k [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/270 [00:00<?, ?B/s]

2_Dense/config.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/3.15M [00:00<?, ?B/s]

...

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.42M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/2.41k [00:00<?, ?B/s]

modules.json:   0%|          | 0.00/461 [00:00<?, ?B/s]

load INSTRUCTOR_Transformer
max_seq_length  512


In [5]:
# Reading dataset from xml files
# Generating dictionaries of sections and slides

import xml.etree.ElementTree as ET
import glob, os

def get_section_details(file):
  tree = ET.parse(file[0])
  root = tree.getroot()

  sections = []

  for neighbor in root.findall('.//{http://www.tei-c.org/ns/1.0}div'):
    i=0
    section_temp = {}
    content=[]
    for head in neighbor:
      if (i==0):
        # print('Section Title: ', head.text,"\n")
        section_temp['title'] = head.text
        # print("i = ",i)
      else:
        content.append(head.text)
      i+=1
    section_temp['content'] = content
    if(content!=[]):
      sections.append(section_temp)
  return sections

def get_slide_details(file):
  tree = ET.parse(file[0])
  root = tree.getroot()

  slides = []
  slide_count=0

  for slide in root.findall('div'):
    slide_count+=1
    slide_temp = {}
    slide_temp['id'] = slide_count

    i=0
    content=[]
    for head in slide:
      if (i==0):
        # print('Section Title: ', head.text,"\n")
        slide_temp['title'] = head.text
      else:
        content.append(head.text)
      i+=1
    slide_temp['content'] = content
    slides.append(slide_temp)

  return slides



In [25]:
# Information Retrieval for Dataset Generation

import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
import json

def gen_query_for_slide(slides,slide_index=0):
  # slide_index = 7
  query = []
  query_instruction = 'Represent the scientific texts for retrieving supporting documents:'
  query.append(query_instruction)
  slide_content = ' '.join(slides[slide_index]['content'])
  slide_title = 'title: '+ slides[slide_index]['title']+ ', content: '
  query.append(slide_title+slide_content)

  return query

# Section as one document of the corpus
def gen_section_corpus(sections):
  corpus = []
  corpus_instruction = 'Represent the scientific document for retrieval'
  for section in sections:
    corpus_temp = []
    corpus_temp.append(corpus_instruction)
    all_paragraph = ' '.join(section['content'])
    section_title = 'title: '+ section['title']+', content: '
    corpus_temp.append(section_title+all_paragraph)
    # print(len(all_paragraph.split()))
    corpus.append(corpus_temp)
  return corpus

def retrieve_section_id(corpus,slides,slide_index):
  query = gen_query_for_slide(slides,slide_index)
  query_embeddings = model.encode([query])
  corpus_embeddings = model.encode(corpus)
  similarities = cosine_similarity(query_embeddings,corpus_embeddings)
  # print(similarities)
  # top_k = 3
  # ind = np.argpartition(similarities, -top_k)[-top_k:]
  # print('top_k = ',ind)
  retrieved_doc_id = np.argmax(similarities)
  return retrieved_doc_id, similarities[0][retrieved_doc_id]

def get_section_to_slide_map(sections,slides,sim_thresh=0.9):
  map_slide_index = []
  map_section_index = []
  corpus = gen_section_corpus(sections)
  for slide_index in range(1,len(slides)):
    section_index, sim_score = retrieve_section_id(corpus,slides,slide_index)
    if (sim_score>sim_thresh):
      map_slide_index.append(slide_index)
      map_section_index.append(section_index)


    # print('slide_index = ',slide_index, 'Matched section index = ', section_index, 'similarity score', sim_score)

  # Map dict has section indices as keys and for every key, the list of corresponding slide indices
  # are saved as values
  map_dict = {k: None for k in sorted(map_section_index)}
  for k in map_dict.keys():
    ind = np.argwhere(map_section_index==k)
    slide_indices_for_k = [map_slide_index[id[0]] for id in ind]
    map_dict[k] = slide_indices_for_k
  return map_dict

In [26]:
paper_index = 6

dir_name = '/content/drive/MyDrive/dataset_0_99/'+str(paper_index)+'/'

prefix = 'Paper'
file_paper = glob.glob(os.path.join(dir_name, prefix + '*.xml'))

prefix = 'slide'
file_slide = glob.glob(os.path.join(dir_name, prefix + '*.xml'))

sections = get_section_details(file_paper)
slides = get_slide_details(file_slide)

# Retrieving section to corresponding slide matching
sim_thresh = 0.91

map_dict = get_section_to_slide_map(sections,slides,sim_thresh)

print(map_dict)

{0: [1], 1: [8, 12], 2: [2], 3: [3, 4], 4: [6], 6: [7], 7: [10, 11], 10: [14, 15], 11: [16], 13: [18], 14: [19], 15: [20]}


In [24]:
print(map_dict)

{0: [1], 2: [2], 3: [3, 4], 4: [6], 6: [7], 1: [8, 12], 7: [10, 11], 10: [14, 15], 11: [16], 13: [18], 14: [19], 15: [20]}
